In [2]:
import numpy as np
import pandas as pd
from datasets import load_from_disk
from datasets import Dataset
import openai
import pickle
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Users/syadav897/python_venv/labweek_v9/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Load openai key
openai.api_key = open("key.txt", "r").read()

In [9]:
embedding_model = 'text-embedding-ada-002'
chat_gpt_model = 'gpt-3.5-turbo'

#### Load PDF File and create chuncks

In [4]:
loader = PyPDFLoader('./SpaceX_NASA_CRS-5_PressKit.pdf')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
split = text_splitter.split_documents(documents)

In [6]:
print("Number of chunks: ", len(split))

Number of chunks:  40


#### Store formed chuncks to disk 

In [8]:
chunks = []
for i in range(len(split)):
    chunks.append(split[i].page_content)

with open('chunks.pkl', 'wb') as f:
    pickle.dump(chunks, f)

#### Create Embeddings for chunck and build faiss index


In [10]:
## load chuncks from pickle file
with open('chunks.pkl', 'rb') as f:
    chunks = pickle.load(f)

In [11]:
data_df = pd.DataFrame(chunks, columns=['text'])
data_df['embedding'] = data_df['text'].apply(lambda x: np.array(openai.Embedding.create(input=x,model=embedding_model)['data'][0]['embedding']))
hf_dataset = Dataset.from_pandas(data_df)
hf_dataset.add_faiss_index(column='embedding')
hf_dataset.save_faiss_index('embedding','faiss_index.faiss')
hf_dataset.drop_index('embedding')
hf_dataset.save_to_disk('./hf_dataset')

100%|██████████| 1/1 [00:00<00:00, 350.02it/s]


#### Chatgpt API for QA and prompt setup

In [14]:
def read(question,passage):
    messages = [
        dict(
            role = 'system',
            content = "you are an intelligent question-answering assistant, you will answer the questions based on context text fed to you."
        )
    ]

    messages.append(dict(role='user',content=passage))

    messages.append(
            dict(
                role="user",
                content=f"\n\n Based on the above text answer this question \n\n, question: {question}",
            )
        )
    
    response = openai.ChatCompletion.create(
            model=chat_gpt_model, messages=messages
        )

    return response["choices"][0]["message"]["content"]

#### Retrieving top chuncks based on question embeddings

In [15]:
## Load the dataset
hf_dataset = load_from_disk('./hf_dataset')
hf_dataset.load_faiss_index('embedding','faiss_index.faiss')


In [16]:
def retriever(question):
    question_embedding = np.array(openai.Embedding.create(input=question,model=embedding_model)['data'][0]['embedding'])
    scores,samples = hf_dataset.get_nearest_examples('embedding',question_embedding,k=5)
    samples_df = pd.DataFrame.from_dict(samples)
    samples_df['scores'] = scores
    samples_df = samples_df.sort_values(by='scores',ascending=True)
    context = '\n'.join(samples_df['text'].tolist())
    return context

In [17]:
question = "what is 2005 NASA Authorization Act"
passage = retriever(question)
answer = read(question,passage)
print(answer)

The 2005 NASA Authorization Act designated the U.S. segment of the space station as a national laboratory.


In [18]:
question = "How many employees does SpaceX have?"
passage = retriever(question)
answer = read(question,passage)
print(answer)

SpaceX has more than 3,000 employees according to the provided information.


In [19]:
question = "where is the headquarters of SpaceX"
passage = retriever(question)
answer = read(question,passage)
print(answer)

The headquarters of SpaceX is located in Hawthorne, California.


In [20]:
question = "List all countries that were involved in this mission"
passage = retriever(question)
answer = read(question,passage)
print(answer)

The countries involved in this mission are the United States, Canada, Japan, Russia, Belgium, Denmark, France, Germany, Italy, the Netherlands, Norway, Spain, Sweden, Switzerland, and the United Kingdom.


In [21]:
question = "List all milestones of spaceX"
passage = retriever(question)
answer = read(question,passage)
print(answer)

- March 2002: SpaceX is incorporated
- March 2006: First flight of SpaceX's Falcon 1 rocket
- August 2006: NASA awards SpaceX $278 million to demonstrate delivery and return of cargo to ISS
- September 2008: Falcon 1, SpaceX's prototype rocket, is the first privately developed liquid-fueled rocket to orbit Earth
- December 2008: NASA awards SpaceX $1.6 billion contract for 12 ISS cargo resupply flights
- July 2009: Falcon 1 becomes the first privately developed rocket to deliver a commercial satellite into orbit
- June 2010: First flight of SpaceX's Falcon 9 rocket, which successfully achieves Earth orbit
- December 2010: SpaceX becomes the first commercial company to launch a spacecraft into orbit and recover it successfully
- May 2012: SpaceX's Dragon becomes the first commercial spacecraft to attach to the ISS, deliver cargo, and return to Earth
- August 2012: SpaceX wins $440 million NASA Space Act Agreement to develop Dragon to transport humans into space
- October 2012: SpaceX co

In [22]:
## let's ask question outside the document
## it should answer that this information is not present in the document

question = "where is isro located"
passage = retriever(question)
answer = read(question,passage)
print(answer)

The location of ISRO (Indian Space Research Organization) is not mentioned in the given text.
